# 0_num_of_patients.ipynb

In [ ]:
import os
import sys
import json
import pathlib
sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import timedelta
import traceback

current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
with open(parent_dir.joinpath("config.json")) as file:
    cfg = json.load(file)

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

driver = cfg["dbms"]
db_cfg = cfg[driver]
username = db_cfg["@user"]
password = db_cfg["@password"]
host = db_cfg["@server"]
port = db_cfg["@port"]
database = db_cfg["@database"]
if driver == "mssql":
    driver = "mssql+pymssql"
elif driver == "postgresql":
    driver = "postgresql+psycopg2"
url = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(url, echo=False)
sessionlocal = sessionmaker(autocommit=False, autoflush=True, bind=engine)

In [ ]:
drug_order = ["Acetaminophen", "Vancomycin", "Naproxen", "Celecoxib", "Acyclovir"]

In [ ]:
def executeQuerynfetchall(engine, sql_query):
    """
        SQL 쿼리 실행 후 결과를 반환하는 함수
    """
    result = None
    try:
        with engine.connect() as conn:
            result = conn.execute(sql_query)
            result = result.fetchall()
        # engine.commit()
    except:
        traceback.print_exc()
    return result

In [ ]:
def get_result_of_cdm_num_of_patients(engine):
    SQL = """SELECT COUNT(DISTINCT p.person_id) as n_patients FROM {@cdm_database_schema}.person p;"""
    sql_param_dict = {}
    sql_param_dict["@cdm_database_schema"] = db_cfg["@cdm_database_schema"]
    query = SQL.format(**sql_param_dict)
    result = executeQuerynfetchall(engine, query)
    if not result:
        print(f"person table is not found in the database.")
        return dict()
    return result[0][0]
def get_result_of_num_patients(engine, drug_name):
    SQL = """SELECT p.is_abnormal, COUNT(DISTINCT p.person_id) as n_patients
    FROM (SELECT *, CASE WHEN n_diff IS NULL THEN 'False' ELSE 'True' END as is_abnormal FROM {@person_database_schema}.{@target_person_table}) p
    GROUP BY p.is_abnormal;"""
    sql_param_dict = {}
    sql_param_dict["@person_database_schema"] = db_cfg["@person_database_schema"]
    sql_param_dict["@target_person_table"] = f"person_{drug_name}"
    query = SQL.format(**sql_param_dict)
    result = executeQuerynfetchall(engine, query)
    if not result:
        print(f"{drug_name} is not found in the database.")
        return dict()
    return {i[0]:i[1] for i in result}

In [ ]:
result_dir = current_dir.joinpath("result")
result_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
number_of_patients_dict = {}
number_of_patients_dict["n_total_patients"] = get_result_of_cdm_num_of_patients(engine)
for drug_name in tqdm(drug_order):
    drug_name = drug_name.lower()
    number_of_patients_dict[drug_name] = get_result_of_num_patients(engine, drug_name)

with open(result_dir.joinpath("number_of_patients.json"), "w") as file:
    json.dump(number_of_patients_dict, file, indent=4)